<a href="https://colab.research.google.com/github/PintoPaola/Inteligencia-Artificial/blob/main/Final/final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np
import pandas as pd

class Board:
    def __init__(self, rows=3, cols=3):
        self.rows = rows
        self.cols = cols
        self.state = self.generate_puzzle()  # Inicializa el estado del tablero aleatorio
        self.goal_state = self.generate_goal()  # Inicializa el estado objetivo del tablero

    def generate_puzzle(self):
        state = np.arange(self.rows * self.cols)  # Crea una secuencia del tamaño del tablero
        np.random.shuffle(state)  # Mezcla aleatoriamente la secuencia
        return state.reshape((self.rows, self.cols))  # Devuelve el estado como una matriz reshaped

    def generate_goal(self):
        return np.append(np.arange(1, self.rows * self.cols), 0).reshape((self.rows, self.cols))
        # Devuelve el estado objetivo del tablero, que es una secuencia ordenada

    def valid_moves(self):
        moves = []
        zero_pos = np.argwhere(self.state == 0)[0]  # Posición del espacio vacío (0)
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Direcciones posibles: arriba, abajo, izquierda, derecha
        for d in directions:
            new_pos = zero_pos + d
            if 0 <= new_pos[0] < self.rows and 0 <= new_pos[1] < self.cols:
                moves.append((zero_pos[0], zero_pos[1], new_pos[0], new_pos[1]))
        return moves
        # Devuelve una lista de movimientos válidos desde la posición del espacio vacío

    def update(self, row1, col1, row2, col2):
        self.state[row1, col1], self.state[row2, col2] = self.state[row2, col2], self.state[row1, col1]
        # Actualiza el tablero intercambiando dos posiciones dadas

    def is_game_over(self):
        return np.array_equal(self.state, self.goal_state)
        # Verifica si el estado actual del tablero es igual al estado objetivo

    def reset(self):
        self.state = self.generate_puzzle()
        # Resetea el tablero generando un nuevo estado aleatorio

class Agent:
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.q_table = {}  # Tabla Q: estado -> valor de acción
        self.alpha = alpha  # Tasa de aprendizaje
        self.prob_exp = prob_exp  # Probabilidad de exploración

    def reset(self):
        pass
        # Método de reinicio del agente, en este caso no realiza ninguna operación

    def move(self, board):
        valid_moves = board.valid_moves()  # Obtiene los movimientos válidos del tablero
        if not valid_moves:
            return None  # Si no hay movimientos válidos, devuelve None

        if np.random.uniform(0, 1) < self.prob_exp:
            # Exploración: elige una acción aleatoria entre los movimientos válidos
            return valid_moves[np.random.choice(len(valid_moves))]
        else:
            # Explotación: elige la acción con el mayor valor de acción según la tabla Q
            max_value = -np.inf
            best_move = None
            for move in valid_moves:
                state_key = tuple(board.state.flatten())  # Convierte el estado del tablero en una tupla hashable
                if state_key not in self.q_table:
                    self.q_table[state_key] = {}  # Crea una entrada para el estado en la tabla Q si no existe
                if move not in self.q_table[state_key]:
                    self.q_table[state_key][move] = 0  # Inicializa el valor de acción si no está definido

                if self.q_table[state_key][move] >= max_value:
                    max_value = self.q_table[state_key][move]  # Actualiza el máximo valor encontrado
                    best_move = move  # Guarda la mejor acción encontrada

            return best_move  # Devuelve la mejor acción encontrada

if __name__ == "__main__":
    agent = Agent(prob_exp=0.2)  # Ejemplo: probabilidad de exploración del 20%
    board = Board()  # Crea un tablero

    rounds = 1000  # Aumenta el número de rondas de entrenamiento

    for _ in range(rounds):
        move = agent.move(board)  # El agente decide el siguiente movimiento
        if move is None:
            board.reset()  # Resetea el tablero si no hay movimientos válidos (opcional)
            continue  # Continúa con la siguiente ronda

        board.update(move[0], move[1], move[2], move[3])  # Actualiza el tablero con el movimiento elegido

    # Generación de la tabla Q
    q_table_entries = []
    for state_key, action_values in agent.q_table.items():
        for action, value in action_values.items():
            q_table_entries.append({'estado': state_key, 'acción': action, 'valor': value})

    tabla_q = pd.DataFrame(q_table_entries)  # Crea un DataFrame de Pandas con los datos de la tabla Q
    print("\nTabla Q :")
    print(tabla_q)  # Imprime la tabla Q completa después de más rondas de entrenamiento




Tabla Q :
                          estado        acción  valor
0    (3, 6, 5, 4, 1, 8, 2, 0, 7)  (2, 1, 1, 1)      0
1    (3, 6, 5, 4, 1, 8, 2, 0, 7)  (2, 1, 2, 0)      0
2    (3, 6, 5, 4, 1, 8, 2, 0, 7)  (2, 1, 2, 2)      0
3    (3, 6, 5, 4, 1, 8, 2, 7, 0)  (2, 2, 1, 2)      0
4    (3, 6, 5, 4, 1, 8, 2, 7, 0)  (2, 2, 2, 1)      0
..                           ...           ...    ...
302  (2, 4, 8, 3, 0, 7, 1, 6, 5)  (1, 1, 1, 0)      0
303  (2, 4, 8, 3, 0, 7, 1, 6, 5)  (1, 1, 1, 2)      0
304  (2, 4, 8, 3, 7, 0, 1, 6, 5)  (1, 2, 0, 2)      0
305  (2, 4, 8, 3, 7, 0, 1, 6, 5)  (1, 2, 2, 2)      0
306  (2, 4, 8, 3, 7, 0, 1, 6, 5)  (1, 2, 1, 1)      0

[307 rows x 3 columns]
